In [1]:
# %%capture
# %pip install datasets
# %pip install transformers

## Libraries and Dependencies

In [1]:
import pandas as pd
import numpy as np
import torch
import nltk
from nltk.tokenize import sent_tokenize
from datasets import load_dataset
from tqdm.notebook import tqdm

# The models the authors used:
from transformers import BertForMaskedLM, BertTokenizer
from transformers import AlbertForMaskedLM, AlbertTokenizer

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Liora\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

## Algorithm Implementation

In [3]:
def mask_sentence(sentence, mask_token, i, M, L_min):
    return [mask_token
            if (j - i) % M == 0
            and (len(sentence[j]) >= L_min
                 or sentence[j].startswith('##')
                 or sentence[min(j+1, len(sentence)-1)].startswith('##'))
            else sentence[j]
            for j in range(len(sentence))]

In [31]:
def BLANC_help(sentence, translation, model, tokenizer, M=6, L_min=4, sep='[SEP]', device='cpu'):
    """
    Calculates BLANC score between a given sentence and its translation using a specified model.

    Parameters:
    - sentence (List[str]): A tokenized sentence.
    - translation (List[str]): The tokenized translation.
    - model: BERT-type model
    - tokenizer: The tokenizer associated with the model used.
    - M (int): Parameter M for the algorithm (default is 6).
    - L_min (int): Minimum length requirement for masked words (default is 4).
    - sep (str): Separator between the inference help (filler/summary) and a sentence from the text (default is '[SEP]').

    Returns:
    - float: BLANC score for the given sentence and its translation.
    """

    filler = ['.'] * len(translation)
    S = [[0, 0], [0, 0]]

    for i in range(M):
        masked_sentence = mask_sentence(sentence, tokenizer.mask_token, i, M, L_min)
        print(masked_sentence)

        input_base = filler + [sep] + masked_sentence
        input_help = translation + [sep] + masked_sentence

        tokenized_input_base = torch.tensor(tokenizer.convert_tokens_to_ids(input_base)).to(device) # Shape: [sequence_length]
        tokenized_input_help = torch.tensor(tokenizer.convert_tokens_to_ids(input_help)).to(device) # Shape: [sequence_length]

        out_base = model(input_ids=tokenized_input_base.unsqueeze(0)).logits  # Shape: [1, sequence_length, model_vocab_size]
        out_help = model(input_ids=tokenized_input_help.unsqueeze(0)).logits  # Shape: [1, sequence_length, model_vocab_size]

        out_base = torch.argmax(out_base.squeeze(0), dim=-1)  # Shape: [sequence_length]
        out_help = torch.argmax(out_help.squeeze(0), dim=-1)  # Shape: [sequence_length]

        masked_tokens = [idx for idx, word in enumerate(masked_sentence) if word == tokenizer.mask_token]

        for j in masked_tokens:
            idx = len(translation + [sep]) + j
            predicted_word_base = tokenizer.convert_ids_to_tokens(out_base[idx].item())
            predicted_word_help = tokenizer.convert_ids_to_tokens(out_help[idx].item())

            print(f'predicted_word_base[{idx - len(translation + [sep])}]: {predicted_word_base}')
            print(f'predicted_word_help[{idx - len(translation + [sep])}]: {predicted_word_help}')
            print(f'sentence[{j}]: {sentence[j]}')

            k = int(predicted_word_base == sentence[j])
            m = int(predicted_word_help == sentence[j])
            S[k][m] += 1


    B = (S[0][1] - S[1][0]) / (S[0][0] + S[1][1] + S[0][1] + S[1][0])

    return B

## Datasets

In [5]:
news_commentary_ds = load_dataset('news_commentary', 'en-fr', split='train')
news_commentary_ds

Generating train split:   0%|          | 0/209479 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'translation'],
    num_rows: 209479
})

In [6]:
parallel_df = pd.DataFrame(news_commentary_ds['translation'][:300])
parallel_df

,en,fr
0,"$10,000 Gold?",L’or à 10.000 dollars l’once ?
1,SAN FRANCISCO – It has never been easy to have...,SAN FRANCISCO – Il n’a jamais été facile d’avo...
2,"Lately, with gold prices up more than 300% ove...","Et aujourd’hui, alors que le cours de l’or a a..."
3,"Just last December, fellow economists Martin F...","En décembre dernier, mes collègues économistes..."
4,Wouldn’t you know it?,Mais devinez ce qui s’est passé ?
...,...,...
295,Although Abdullah is usually referred to in th...,Bien qu'Abdallah soit généralement considéré à...
296,"The Sudairis, it seems, have apparently left t...",Ils semblent avoir laissé leur demi-frère se c...
297,For although Crown Prince Abdullah has his own...,Même si le prince héritier Abdallah bénéficie ...
298,The idea of normalizing relations with Israel ...,L'idée d'une normalisation des relations avec ...


## Model and Tokenizer

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)
model = BertForMaskedLM.from_pretrained('bert-base-uncased').to(DEVICE)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Preprocessing

In [8]:
sentences = [tokenizer.tokenize(sentence)
             for sentence in parallel_df['en']]  # (List[List[str]])

translations = [tokenizer.tokenize(translation)
                for translation in parallel_df['fr']] # (List[List[str]])

In [28]:
import random
random.seed(42)

def random_words_translation(translation):
    random_translation = ""
    translation = translation.split()
    for _ in range(len(translation)):
        random_translation += random.choice(translation) + ' '
    return random_translation

random_translations = [random_words_translation(translation) for translation in parallel_df['fr'].unique()]
tokenized_random_translations = [tokenizer.tokenize(translation) for translation in random_translations]


Abba Eban : La Voix d\u0027Israel
Abba Eban: The Voice of Israel
['ab', '##ba', 'e', '##ban', ':', 'la', 'vo', '##ix', 'd', '\\', 'u', '##00', '##27', '##is', '##rae', '##l']
['ab', '##ba', 'e', '##ban', ':', 'the', 'voice', 'of', 'israel']


In [10]:
# print(f'longest sentence: {max([len(sent) for sent in sentences])}')
# print(f'longest translation: {max([len(sent) for sent in translations])}')

## Running the Program

In [9]:
BLANC_help(translations[0], sentences[0], model, tokenizer, device=DEVICE)

0.0

In [12]:
%%time
scores = [BLANC_help(translation, sentence, model, tokenizer, device=DEVICE)
          for translation, sentence in zip(translations, sentences)]

CPU times: total: 30.6 s
Wall time: 1min 12s


In [29]:
scores

[0.0,
 0.05263157894736842,
 0.037037037037037035,
 0.03508771929824561,
 0.0,
 0.0,
 0.0,
 -0.05555555555555555,
 0.046511627906976744,
 0.0,
 0.06666666666666667,
 0.05,
 0.0,
 0.09523809523809523,
 0.043478260869565216,
 0.0,
 0.0,
 0.0,
 -0.041666666666666664,
 0.0,
 0.0,
 0.10606060606060606,
 -0.045454545454545456,
 0.09090909090909091,
 0.047619047619047616,
 0.0,
 0.058823529411764705,
 0.05405405405405406,
 0.06896551724137931,
 -0.029411764705882353,
 -0.05128205128205128,
 0.027777777777777776,
 -0.04,
 0.0,
 0.0,
 -0.07692307692307693,
 -0.047619047619047616,
 0.0,
 -0.029411764705882353,
 0.0,
 -0.07142857142857142,
 0.034482758620689655,
 0.025,
 -0.03225806451612903,
 0.05555555555555555,
 0.0,
 0.0,
 0.06666666666666667,
 0.023809523809523808,
 0.05,
 0.0,
 -0.041666666666666664,
 -0.03571428571428571,
 0.1111111111111111,
 0.0,
 0.0,
 0.07142857142857142,
 0.023255813953488372,
 0.0,
 0.0392156862745098,
 0.09090909090909091,
 0.02040816326530612,
 0.01818181818181818,

In [38]:
#Bad results 1

print("translation : ", parallel_df['fr'].iloc[71])
print("sentence : ", parallel_df['en'].iloc[71])
print("tokenization of trans. : ", translations[71])
print("tokenization of sentence : ",sentences[71])
print('\n')

BLANC_help(translations[71], sentences[71], model, tokenizer, device=DEVICE)

253 0.3333333333333333
translation :  Quand les riches deviennent moins riches, les pauvres deviennent encore plus pauvres.
sentence :  When the rich get less rich, the poor get poorer.
tokenization of trans. :  ['quan', '##d', 'les', 'riches', 'devi', '##enne', '##nt', 'moi', '##ns', 'riches', ',', 'les', 'pau', '##vres', 'devi', '##enne', '##nt', 'encore', 'plus', 'pau', '##vres', '.']
tokenization of sentence :  ['when', 'the', 'rich', 'get', 'less', 'rich', ',', 'the', 'poor', 'get', 'poorer', '.']


['[MASK]', '##d', 'les', 'riches', 'devi', '##enne', '[MASK]', 'moi', '##ns', 'riches', ',', 'les', '[MASK]', '##vres', 'devi', '##enne', '##nt', 'encore', '[MASK]', 'pau', '##vres', '.']
predicted_word_base[0]: quan
predicted_word_help[0]: quan
sentence[0]: quan
predicted_word_base[6]: ##nt
predicted_word_help[6]: ##nt
sentence[6]: ##nt
predicted_word_base[12]: pau
predicted_word_help[12]: pau
sentence[12]: pau
predicted_word_base[18]: les
predicted_word_help[18]: les
sentence[18]: pl

-0.1111111111111111

In [40]:
#Bad results 2
print(scores.index(-0.25))

print("translation : ", parallel_df['fr'].iloc[156])
print("sentence : ", parallel_df['en'].iloc[156])
print("tokenization of trans. : ", translations[156])
print("tokenization of sentence : ",sentences[156])
print('\n')

BLANC_help(translations[156], sentences[156], model, tokenizer, device=DEVICE)

156
translation :  Le 11 septembre en perspective
sentence :  9/11 in Perspective
tokenization of trans. :  ['le', '11', 'sept', '##em', '##bre', 'en', 'perspective']
tokenization of sentence :  ['9', '/', '11', 'in', 'perspective']


['le', '11', 'sept', '##em', '##bre', 'en', '[MASK]']
predicted_word_base[6]: france
predicted_word_help[6]: france
sentence[6]: perspective
['le', '11', 'sept', '##em', '##bre', 'en', 'perspective']
['le', '11', '[MASK]', '##em', '##bre', 'en', 'perspective']
predicted_word_base[2]: sept
predicted_word_help[2]: nov
sentence[2]: sept
['le', '11', 'sept', '[MASK]', '##bre', 'en', 'perspective']
predicted_word_base[3]: ##em
predicted_word_help[3]: ##em
sentence[3]: ##em
['le', '11', 'sept', '##em', '[MASK]', 'en', 'perspective']
predicted_word_base[4]: ##bre
predicted_word_help[4]: ##bre
sentence[4]: ##bre
['le', '11', 'sept', '##em', '##bre', 'en', 'perspective']


-0.25

In [43]:
#Good results 1
print(np.argmax(scores), max(scores))

print("translation : ", parallel_df['fr'].iloc[253])
print("sentence : ", parallel_df['en'].iloc[253])
print("tokenization of trans. : ", translations[253])
print("tokenization of sentence : ",sentences[253])
print('\n')

BLANC_help(translations[253], sentences[253], model, tokenizer, device=DEVICE)

253 0.3333333333333333
translation :  Abba Eban : La Voix d\u0027Israel
sentence :  Abba Eban: The Voice of Israel
tokenization of trans. :  ['ab', '##ba', 'e', '##ban', ':', 'la', 'vo', '##ix', 'd', '\\', 'u', '##00', '##27', '##is', '##rae', '##l']
tokenization of sentence :  ['ab', '##ba', 'e', '##ban', ':', 'the', 'voice', 'of', 'israel']


['[MASK]', '##ba', 'e', '##ban', ':', 'la', '[MASK]', '##ix', 'd', '\\', 'u', '##00', '[MASK]', '##is', '##rae', '##l']
predicted_word_base[0]: e
predicted_word_help[0]: ab
sentence[0]: ab
predicted_word_base[6]: vo
predicted_word_help[6]: vo
sentence[6]: vo
predicted_word_base[12]: ##0
predicted_word_help[12]: ##0
sentence[12]: ##27
['ab', '[MASK]', 'e', '##ban', ':', 'la', 'vo', '[MASK]', 'd', '\\', 'u', '##00', '##27', '[MASK]', '##rae', '##l']
predicted_word_base[1]: -
predicted_word_help[1]: ##ba
sentence[1]: ##ba
predicted_word_base[7]: ##ix
predicted_word_help[7]: ##ix
sentence[7]: ##ix
predicted_word_base[13]: ##z
predicted_word_help[13]

0.3333333333333333

In [42]:
# Good result 2
print(scores.index(0.25))

print("translation : ", parallel_df['fr'].iloc[90])
print("sentence : ", parallel_df['en'].iloc[90])
print("tokenization of trans. : ", translations[90])
print("tokenization of sentence : ",sentences[90])
print('\n')

BLANC_help(translations[90], sentences[90], model, tokenizer, device=DEVICE)

90
translation :  Il s’agira d’une caractéristique du consensus de 1945.
sentence :  It was part of the 1945 consensus.
tokenization of trans. :  ['il', 's', '’', 'ag', '##ira', 'd', '’', 'une', 'cara', '##cter', '##ist', '##ique', 'du', 'consensus', 'de', '1945', '.']
tokenization of sentence :  ['it', 'was', 'part', 'of', 'the', '1945', 'consensus', '.']


['il', 's', '’', 'ag', '##ira', 'd', '’', 'une', 'cara', '##cter', '##ist', '##ique', 'du', 'consensus', 'de', '1945', '.']
['il', 's', '’', 'ag', '##ira', 'd', '’', 'une', 'cara', '##cter', '##ist', '##ique', 'du', '[MASK]', 'de', '1945', '.']
predicted_word_base[13]: year
predicted_word_help[13]: year
sentence[13]: consensus
['il', 's', '’', 'ag', '##ira', 'd', '’', 'une', '[MASK]', '##cter', '##ist', '##ique', 'du', 'consensus', 'de', '1945', '.']
predicted_word_base[8]: ba
predicted_word_help[8]: cara
sentence[8]: cara
['il', 's', '’', '[MASK]', '##ira', 'd', '’', 'une', 'cara', '[MASK]', '##ist', '##ique', 'du', 'consensus', '

0.25

In [45]:
# Null result
print(scores.index(0.0))

print("translation : ", parallel_df['fr'].iloc[0])
print("sentence : ", parallel_df['en'].iloc[0])
print("tokenization of trans. : ", translations[0])
print("tokenization of sentence : ",sentences[0])
print('\n')

BLANC_help(translations[0], sentences[0], model, tokenizer, device=DEVICE)

0
translation :  L’or à 10.000 dollars l’once ?
sentence :  $10,000 Gold?
tokenization of trans. :  ['l', '’', 'or', 'a', '10', '.', '000', 'dollars', 'l', '’', 'once', '?']
tokenization of sentence :  ['$', '10', ',', '000', 'gold', '?']


['l', '’', 'or', 'a', '10', '.', '000', 'dollars', 'l', '’', 'once', '?']
['l', '’', 'or', 'a', '10', '.', '000', '[MASK]', 'l', '’', 'once', '?']
predicted_word_base[7]: -
predicted_word_help[7]: gold
sentence[7]: dollars
['l', '’', 'or', 'a', '10', '.', '000', 'dollars', 'l', '’', 'once', '?']
['l', '’', 'or', 'a', '10', '.', '000', 'dollars', 'l', '’', 'once', '?']
['l', '’', 'or', 'a', '10', '.', '000', 'dollars', 'l', '’', '[MASK]', '?']
predicted_word_base[10]: ##y
predicted_word_help[10]: ##y
sentence[10]: once
['l', '’', 'or', 'a', '10', '.', '000', 'dollars', 'l', '’', 'once', '?']


0.0

In [49]:
# Low score 1

print("translation : ", parallel_df['fr'].iloc[2])
print("sentence : ", parallel_df['en'].iloc[2])
print("tokenization of trans. : ", translations[2])
print("tokenization of sentence : ",sentences[2])
print('\n')

BLANC_help(translations[2], sentences[2], model, tokenizer, device=DEVICE)

translation :  Et aujourd’hui, alors que le cours de l’or a augmenté de 300 pour cent au cours de la dernière décennie, c’est plus difficile que jamais.
sentence :  Lately, with gold prices up more than 300% over the last decade, it is harder than ever.
tokenization of trans. :  ['et', 'au', '##jou', '##rd', '’', 'hui', ',', 'al', '##ors', 'que', 'le', 'co', '##urs', 'de', 'l', '’', 'or', 'a', 'aug', '##ment', '##e', 'de', '300', 'pour', 'cent', 'au', 'co', '##urs', 'de', 'la', 'der', '##nier', '##e', 'dec', '##en', '##nie', ',', 'c', '’', 'est', 'plus', 'di', '##ffi', '##ci', '##le', 'que', 'jam', '##ais', '.']
tokenization of sentence :  ['lately', ',', 'with', 'gold', 'prices', 'up', 'more', 'than', '300', '%', 'over', 'the', 'last', 'decade', ',', 'it', 'is', 'harder', 'than', 'ever', '.']


['et', 'au', '##jou', '##rd', '’', 'hui', ',', 'al', '##ors', 'que', 'le', 'co', '[MASK]', 'de', 'l', '’', 'or', 'a', '[MASK]', '##ment', '##e', 'de', '300', 'pour', '[MASK]', 'au', 'co', '##ur

0.037037037037037035

In [50]:
# Low score 2

print("translation : ", parallel_df['fr'].iloc[1])
print("sentence : ", parallel_df['en'].iloc[1])
print("tokenization of trans. : ", translations[1])
print("tokenization of sentence : ",sentences[1])
print('\n')

BLANC_help(translations[1], sentences[1], model, tokenizer, device=DEVICE)

translation :  SAN FRANCISCO – Il n’a jamais été facile d’avoir une discussion rationnelle sur la valeur du métal jaune.
sentence :  SAN FRANCISCO – It has never been easy to have a rational conversation about the value of gold.
tokenization of trans. :  ['san', 'francisco', '–', 'il', 'n', '’', 'a', 'jam', '##ais', 'et', '##e', 'fa', '##ci', '##le', 'd', '’', 'av', '##oir', 'une', 'discussion', 'ratio', '##nne', '##lle', 'sur', 'la', 'vale', '##ur', 'du', 'metal', 'ja', '##une', '.']
tokenization of sentence :  ['san', 'francisco', '–', 'it', 'has', 'never', 'been', 'easy', 'to', 'have', 'a', 'rational', 'conversation', 'about', 'the', 'value', 'of', 'gold', '.']


['san', 'francisco', '–', 'il', 'n', '’', 'a', 'jam', '##ais', 'et', '##e', 'fa', '[MASK]', '##le', 'd', '’', 'av', '##oir', 'une', 'discussion', 'ratio', '##nne', '##lle', 'sur', 'la', 'vale', '##ur', 'du', 'metal', 'ja', '[MASK]', '.']
predicted_word_base[12]: ##ci
predicted_word_help[12]: ##ci
sentence[12]: ##ci
predicte

0.05263157894736842

In [14]:
data = {}
data['BLANC_help_300_translation'] = scores

import json

def add_results_to_json(new_data, file_path = "./results.json"):
    try:
        with open(file_path, 'r') as json_file:
            existing_data = json.load(json_file)
    except FileNotFoundError:
        existing_data = {}

    for key, value in new_data.items():
        existing_data[key] = value

    with open(file_path, 'w') as json_file:
        json.dump(existing_data, json_file, indent=2)

    print(f"Data has been added to {file_path}")

add_results_to_json(data)

Data has been added to ./results.json


In [ ]:
import json

def change_key_name(json_file_path, old_key, new_key):
    try:
        # Load existing data from the JSON file
        with open(json_file_path, 'r') as json_file:
            existing_data = json.load(json_file)
    except FileNotFoundError:
        print(f"The file {json_file_path} does not exist.")
        return

    # Check if the old key exists in the data
    if old_key in existing_data:
        # Create a new key with the desired name
        existing_data[new_key] = existing_data.pop(old_key)

        # Write the updated data back to the JSON file
        with open(json_file_path, 'w') as json_file:
            json.dump(existing_data, json_file, indent=2)

        print(f"The key '{old_key}' has been changed to '{new_key}' in {json_file_path}")
    else:
        print(f"The key '{old_key}' does not exist in the data.")

# Example usage:
json_file_path = "./results.json"
old_key_name = "BLANC_help_unbatched_300"
new_key_name = "BLANC_help_300"

change_key_name(json_file_path, old_key_name, new_key_name)


In [51]:
import json

def remove_key_from_json(file_path, key_to_remove):
    try:
        # Load existing data from the JSON file
        with open(file_path, 'r') as json_file:
            existing_data = json.load(json_file)
    except FileNotFoundError:
        print(f"The file {file_path} does not exist.")
        return

    # Check if the key exists in the data
    if key_to_remove in existing_data:
        # Remove the key
        del existing_data[key_to_remove]

        # Write the updated data back to the JSON file
        with open(file_path, 'w') as json_file:
            json.dump(existing_data, json_file, indent=2)

        print(f"The key '{key_to_remove}' has been removed from {file_path}")
    else:
        print(f"The key '{key_to_remove}' does not exist in the data.")

# Example usage:
json_file_path = "./results.json"
key_to_remove = "BLANC_help_300_translation"

remove_key_from_json(json_file_path, key_to_remove)


The key 'BLANC_help_300_translation' has been removed from ./results.json
